In [ ]:
##Import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import openpyxl
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from mpl_toolkits.axes_grid1 import make_axes_locatable 
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = preprocessing.MinMaxScaler() 
from sklearn.decomposition import PCA

In [ ]:
#load data set:
random_state_values = range(1, 500)
df_dataset = pd.read_excel('Dataset.xlsx')


Descriptors = pd.ExcelFile('Descriptors.xlsx')
# Function to load and process descriptors
def load_and_process_descriptors(Descriptors):
    descriptors_df = pd.DataFrame()
    for num in range(12):
        descriptors = pd.read_excel(Descriptors, f'descriptors.xls_{num}')
        descriptors_df = pd.concat([descriptors_df, descriptors], axis=1)
    return descriptors_df.values[:, :]

# Process descriptors
Descriptors= load_and_process_descriptors(Descriptors)

print(Descriptors.shape)
#scale descriptors
descriptors_scale = min_max_scaler.fit_transform(Descriptors)



density= df_dataset.iloc[:, 7].tolist()
Cp= df_dataset.iloc[:, 9].tolist()
Cv= df_dataset.iloc[:, 10].tolist()
Rg=df_dataset.iloc[:, 8].tolist()
bulk=df_dataset.iloc[:, 11].tolist()
linear=df_dataset.iloc[:, 13].tolist()
volume=df_dataset.iloc[:, 12].tolist()
Y = np.array([density, Cp, Cv,Rg,bulk,linear,volume]).T


In [ ]:
# multi-task RF with all descriptors
r2_density_tests = []
r2_Cp_tests = []
r2_Cv_tests = []
r2_Rg_tests = []
r2_bulk_tests = []
r2_linear_tests = []
r2_volume_tests = []

# Function to run the model with specified random states
def run_model(random_state_split_1, random_state_split_2):
    # Split the dataset
    X_train, X_test, Y_train, Y_test = train_test_split(descriptors_scale, 
                                                        Y, test_size=0.2, 
                                                        random_state=random_state_split_1)
    X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, 
                                                    test_size=0.5, 
                                                    random_state=random_state_split_2)

    # Scale the Y values
    scaler = StandardScaler()
    Y_train_scaled = scaler.fit_transform(Y_train)
    Y_val_scaled = scaler.transform(Y_val)
    Y_test_scaled = scaler.transform(Y_test)

    # Define the parameter grid for grid search
    param_grid = {
        'n_estimators': [100, 150, 200, 250, 300, 400, 500],
        'max_depth': [10, 20, 80, 100],
        'max_features': ['sqrt', 'log2'],
        'min_samples_split': [2, 5, 12],
        'min_samples_leaf': [1]
    }

    # Initialize the Random Forest model
    model = RandomForestRegressor(random_state=1)

    # Set up GridSearchCV
    grid_search = GridSearchCV(estimator=model, 
                               param_grid=param_grid, 
                               cv=5, 
                               scoring='r2', 
                               verbose=1, 
                               n_jobs=-1)

    # Fit GridSearchCV to the training data
    grid_search.fit(X_train, Y_train_scaled)

    # Get the best estimator from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions on the training, validation, and test sets
    Y_train_pred = best_model.predict(X_train)
    Y_val_pred = best_model.predict(X_val)
    Y_test_pred = best_model.predict(X_test)

    # Calculate R-squared for each property and store results
    r2_density_test = r2_score(Y_test[:, 0], scaler.inverse_transform(Y_test_pred)[:, 0])
    r2_Cp_test = r2_score(Y_test[:, 1], scaler.inverse_transform(Y_test_pred)[:, 1])
    r2_Cv_test = r2_score(Y_test[:, 2], scaler.inverse_transform(Y_test_pred)[:, 2])
    r2_Rg_test = r2_score(Y_test[:, 3], scaler.inverse_transform(Y_test_pred)[:, 3])
    r2_bulk_test = r2_score(Y_test[:, 4], scaler.inverse_transform(Y_test_pred)[:, 4])
    r2_linear_test = r2_score(Y_test[:, 5], scaler.inverse_transform(Y_test_pred)[:, 5])
    r2_volume_test = r2_score(Y_test[:, 6], scaler.inverse_transform(Y_test_pred)[:, 6])

    # Append R² values to their respective lists
    r2_density_tests.append(r2_density_test)
    r2_Cp_tests.append(r2_Cp_test)
    r2_Cv_tests.append(r2_Cv_test)
    r2_Rg_tests.append(r2_Rg_test)
    r2_bulk_tests.append(r2_bulk_test)
    r2_linear_tests.append(r2_linear_test)
    r2_volume_tests.append(r2_volume_test)

# Run the model with the first set of random states
run_model(97, 346)

# Run the model with the second set of random states
run_model(326, 16)

# Run the model with the third set of random states
run_model(206, 37)

# Calculate and print average R² values for each property
print("\nAverage R² Values Across 3 Models:")
print(f"Average R² for Density: {np.mean(r2_density_tests)}")
print(f"Average R² for Cp: {np.mean(r2_Cp_tests)}")
print(f"Average R² for Cv: {np.mean(r2_Cv_tests)}")
print(f"Average R² for Rg: {np.mean(r2_Rg_tests)}")
print(f"Average R² for Bulk: {np.mean(r2_bulk_tests)}")
print(f"Average R² for Linear: {np.mean(r2_linear_tests)}")
print(f"Average R² for Volume: {np.mean(r2_volume_tests)}")


In [ ]:
#Important feature analysis

random_state_values = range(1, 500)
df_dataset = pd.read_excel('Dataset.xlsx')


Descriptors = pd.ExcelFile('Descriptors.xlsx')
# Function to load and process descriptors
def load_and_process_descriptors(Descriptors):
    descriptors_df = pd.DataFrame()
    for num in range(12):
        descriptors = pd.read_excel(Descriptors, f'descriptors.xls_{num}')
        descriptors_df = pd.concat([descriptors_df, descriptors], axis=1)
    return descriptors_df  # Return the DataFrame

# Process descriptors, keep DataFrame with column names
descriptors_df = load_and_process_descriptors(Descriptors)

# Save the column names before converting to NumPy array
column_names = descriptors_df.columns

# Scale descriptors
min_max_scaler = MinMaxScaler()
descriptors_scale = min_max_scaler.fit_transform(descriptors_df)


density= df_dataset.iloc[:, 7].tolist()
Cp= df_dataset.iloc[:, 9].tolist()
Cv= df_dataset.iloc[:, 10].tolist()
Rg=df_dataset.iloc[:, 8].tolist()
bulk=df_dataset.iloc[:, 11].tolist()
linear=df_dataset.iloc[:, 13].tolist()
volume=df_dataset.iloc[:, 12].tolist()
Y = np.array([density, Cp, Cv,Rg,bulk,linear,volume]).T

X_train, X_test, Y_train, Y_test = train_test_split(descriptors_scale, 
                                                                Y, test_size=0.2, 
                                                                random_state=326)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, 
                                                              test_size=0.5, 
                                                              random_state=16)


# Initialize the model
model = RandomForestRegressor(n_estimators=200, max_depth=80, max_features='sqrt', min_samples_split=2,
                              min_samples_leaf=1, random_state=1)

# Fit the model on the training data
model.fit(X_train, Y_train)

# Plot feature importance
importance = model.feature_importances_
#importance_df = pd.Series(importance, index=X_train.columns)  # Use correct column names
importance_df = pd.Series(importance, index=column_names).sort_values()

fig, ax = plt.subplots(figsize=(10, 6), dpi=200)

importance_df.nlargest(10).plot(kind='barh', color='magenta')
ax.set_xlabel('Feature Importance', fontsize=28,  fontname='sans serif')
ax.set_ylabel('Features', fontsize=28, fontweight='bold', fontname='sans serif')
#plt.title('Top Feature Importances', fontsize=22, fontname='Times New Roman')


plt.show()

In [ ]:
# multi-task RF with important descriptors

top_features = importance_df.nlargest(10).index

# Map the feature names to their respective integer indices
feature_indices = [descriptors_df.columns.get_loc(feature) for feature in top_features]

# Select these top features from the scaled descriptor array
X_top = descriptors_scale[:, feature_indices]

r2_density_tests = []
r2_Cp_tests = []
r2_Cv_tests = []
r2_Rg_tests = []
r2_bulk_tests = []
r2_linear_tests = []
r2_volume_tests = []

# Function to run the model with specified random states
def run_model(random_state_split_1, random_state_split_2):
    # Split the dataset
    X_train, X_test, Y_train, Y_test = train_test_split(descriptors_scale, 
                                                        Y, test_size=0.2, 
                                                        random_state=random_state_split_1)
    X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, 
                                                    test_size=0.5, 
                                                    random_state=random_state_split_2)

    # Scale the Y values
    scaler = StandardScaler()
    Y_train_scaled = scaler.fit_transform(Y_train)
    Y_val_scaled = scaler.transform(Y_val)
    Y_test_scaled = scaler.transform(Y_test)

    # Define the parameter grid for grid search
    param_grid = {
        'n_estimators': [100, 150, 200, 250, 300, 400, 500],
        'max_depth': [10, 20, 80, 100],
        'max_features': ['sqrt', 'log2'],
        'min_samples_split': [2, 5, 12],
        'min_samples_leaf': [1]
    }

    # Initialize the Random Forest model
    model = RandomForestRegressor(random_state=1)

    # Set up GridSearchCV
    grid_search = GridSearchCV(estimator=model, 
                               param_grid=param_grid, 
                               cv=5, 
                               scoring='r2', 
                               verbose=1, 
                               n_jobs=-1)

    # Fit GridSearchCV to the training data
    grid_search.fit(X_train, Y_train_scaled)

    # Get the best estimator from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions on the training, validation, and test sets
    Y_train_pred = best_model.predict(X_train)
    Y_val_pred = best_model.predict(X_val)
    Y_test_pred = best_model.predict(X_test)

    # Calculate R-squared for each property and store results
    r2_density_test = r2_score(Y_test[:, 0], scaler.inverse_transform(Y_test_pred)[:, 0])
    r2_Cp_test = r2_score(Y_test[:, 1], scaler.inverse_transform(Y_test_pred)[:, 1])
    r2_Cv_test = r2_score(Y_test[:, 2], scaler.inverse_transform(Y_test_pred)[:, 2])
    r2_Rg_test = r2_score(Y_test[:, 3], scaler.inverse_transform(Y_test_pred)[:, 3])
    r2_bulk_test = r2_score(Y_test[:, 4], scaler.inverse_transform(Y_test_pred)[:, 4])
    r2_linear_test = r2_score(Y_test[:, 5], scaler.inverse_transform(Y_test_pred)[:, 5])
    r2_volume_test = r2_score(Y_test[:, 6], scaler.inverse_transform(Y_test_pred)[:, 6])

    # Append R² values to their respective lists
    r2_density_tests.append(r2_density_test)
    r2_Cp_tests.append(r2_Cp_test)
    r2_Cv_tests.append(r2_Cv_test)
    r2_Rg_tests.append(r2_Rg_test)
    r2_bulk_tests.append(r2_bulk_test)
    r2_linear_tests.append(r2_linear_test)
    r2_volume_tests.append(r2_volume_test)

run_model(418,437)
run_model(288, 288)
run_model(288, 171)

# Calculate and print average R² values for each property
print("\nAverage R² Values Across 3 Models:")
print(f"Average R² for Density: {np.mean(r2_density_tests)}")
print(f"Average R² for Cp: {np.mean(r2_Cp_tests)}")
print(f"Average R² for Cv: {np.mean(r2_Cv_tests)}")
print(f"Average R² for Rg: {np.mean(r2_Rg_tests)}")
print(f"Average R² for Bulk: {np.mean(r2_bulk_tests)}")
print(f"Average R² for Linear: {np.mean(r2_linear_tests)}")
print(f"Average R² for Volume: {np.mean(r2_volume_tests)}")